In [ ]:
import pathlib
import re
import collections
import functools
import difflib
import operator
import networkx as nx
import warnings
import pickle
warnings.simplefilter('ignore') # for networkx drawing
import matplotlib.pylab
%matplotlib inline
matplotlib.pylab.rcParams["figure.figsize"] = (14, 18)

# Indlæs data
## Hvor er filerne?
Ret nedenstående hvis det køres på ikke-min computer.

In [ ]:
datadir = pathlib.Path("/home/david/pro/scc/data")

Gem for resten alle sete gode matches; de kan alle bruges til træning senere.

In [ ]:
%run utils.py
cache = ApprovedMatches(datadir / "approved.pickle")

Headers:

- 0  amt
- 1  herred
- 2  sogn
- 3  navn
- 4  køn
- 5  fødested
- 6  fødeaar
- 7  civilstand
- 8  position
- 9  erhverv
- 10 husstnr
- 11 kipnr
- 12 løbenr

Bruger namedtuples for at gøre koden senere nemmere. Og de kan hashes, så vi kan have sets og dicts.

Ser på de cleanede `lc_`-filer.

In [ ]:
list(datadir.glob("lc_*.csv"))

## Overvej hvilked område at se på
Bemærk: Det er problematisk ift. træning af model fordi vi netop vil have en, der kan matche folk efter de er flyttet også.  Overvej bedre blocking til stikprøver.

In [ ]:
areas = {}
for fn in sorted(datadir.glob("lc_*.csv")):
    print(fn.stem)
    with fn.open("r", encoding="utf-8") as fd:
        next(fd)
        areas[fn.stem] = collections.Counter(line.split("|")[1] for line in fd)
#for year in sorted(areas):
    #print(areas[year])

Okay tællingen for 1885 er kun for København.  Useless.

In [ ]:
del(areas["lc_FT1885_SDU"]) # only copenhagen, ignore for now

Hvilke områder er i alle (resterende) tællinger?

In [ ]:
in_common = functools.reduce(set.intersection, (set(arealist) for arealist in areas.values()))

Og hvad er mindste antal folk i hvert område (øvre grænse for fællesmængde)?

In [ ]:
{area: min(arealist[area]
           for arealist in areas.values())
 for area in in_common}

## Indlæs data for det valgte afgrænsede område
Bemærk: alt det med rettelse af fødested herunder burde gøres i de cleanede `lc_`-datafiler, ikke her.

For resten så begrænser jeg også lige navne for at få endnu færre rækker; arbejder på min bærbare lige nu.

In [ ]:
subset = []
area = "læsø"
unhandled = set()
for fnstem in areas:
    year = int(re.search(r"\d{4}", fnstem).group(0))
    print(year)
    fn = datadir / (fnstem + ".csv")
    with fn.open("r", encoding="UTF-8") as fd:
        next(fd)
        acc = []
        for line in fd:
            row = line.strip().split("|")
            if not (row[1] == area and row[3].startswith("an")):
                continue
            # fødeår som int
            row[6] = int(row[6])
            # prøv at fikse fødesteder
            fødested = row[5].replace(".","").split()
            if len(fødested) > 1:
                if fødested[-2:] == ["i", "sognet"] or fødested == ["heri", "sognet"] or "her i s" in row[5]: # {her ,}i sognet
                    fødested = row[2] # erstat med sognet
                elif "sogn" in fødested: # ... sogn på læsø / ... sogn ... amt
                    index = fødested.index("sogn")
                    fødested = " ".join(fødested[:index])
                elif "amt" in fødested:
                    index = fødested.index("amt") # ... by amt
                    fødested = " ".join(fødested[:index-1])
                elif fødested[-1].startswith("["): # ... [by]
                    fødested = " ".join(fødested[:-1])
                else:
                    fødested = " ".join(fødested)
                    unhandled.add(fødested)
                row[5] = fødested
            row = Entry(*row)
            acc.append(row)
        subset.append((year, acc))

Der er nogle fødesteder med flere ord, som endnu ikke håndteres.  Det kan vist nemt ordnes, men hører ikke til her (der er rigeligt med spaghetti allerede).

In [ ]:
print(unhandled)

In [ ]:
print([(year, len(data), len(set(data))) for (year, data) in subset])

In [ ]:
len(set(row[3] for row in subset[0][1]))

In [ ]:
(a_year, a_data), (b_year, b_data) = subset[:2]

# Prøv at finde nogle fornuftige matches
## Matches som max-weight matching
Det ligger jo i navnet at det er en god ide.  For de uindviede: <a href="https://en.wikipedia.org/wiki/Matching_(graph_theory)">wiki</a>.

Vil bruge edit distances til vægte - tager fødested $d_p$ og $d_n$ (ratio ens i stedet for direkte edit distance; længere navne kan have flere fejl; ratio går fra 0 til 1 hvor 1 betyder "ens").  Desuden tillader jeg op til 3 års fejl i angivelse af fødselsår (forskel $d_y$; her er det bare absolut forskel (mindre er bedre)).

In [ ]:
diff_name = difflib.SequenceMatcher()
diff_place = difflib.SequenceMatcher()

Det hele er bare et eksperiment.  Nedenfor tilføjes kun kanter hvis $d_y \leq 3 \wedge d_n \geq 0.85$.  Vægte på kanter straffes for alle variabler, men måske med $d_n$ (problem: giver 0 hvis fødested er angivet helt forskelligt, men det kan være okay - se kommentarer til sidst).  Formel for vægt er lige nu:
$$w = d_n^2 \cdot d_p \cdot \frac{1}{1 + \frac{d_n}{3}}$$

In [ ]:
G = nx.Graph()
for i, a_row in enumerate(a_data):
    diff_name.set_seq1(a_row.navn)
    diff_place.set_seq1(a_row.fødested)

    for j, b_row in enumerate(b_data):
        # first filter really bad matches
        age_diff = abs(a_row.fødeår - b_row.fødeår)
        if age_diff > 3:
            continue

        diff_name.set_seq2(b_row.navn)
        ratio_name = diff_name.ratio()
        if ratio_name < 0.85:
            continue
        
        diff_place.set_seq2(b_row.fødested)
        ratio_place = diff_place.ratio()
        
        # if maybe decent match, add edge
        w = ratio_name**2 * ratio_place * 1/(1+age_diff/3)
        G.add_edge("a" + str(i), "b" + str(j), weight=w)

Lad os da lige visualisere grafen.  Det er fancy.

In [ ]:
pos = {}
A = set()
B = set()
for n in G.nodes_iter():
    if n.startswith("a"):
        pos[n] = (0, int(n[1:]))
        A.add(n)
    else:
        pos[n] = (4, int(n[1:]))
        B.add(n)

In [ ]:
for k,v in sorted(nx.degree(G, A).items(), key=operator.itemgetter(1), reverse=True):
    print(k, v)

In [ ]:
match = nx.max_weight_matching(G, maxcardinality=True)
match = set(tuple(sorted(pair)) for pair in match.items())

In [ ]:
dummy = nx.Graph()
for u, v in match:
    dummy.add_edge(u,v)
nx.draw(G, pos=pos, width=0.4, node_size=5)
nx.draw_networkx_edges(dummy, edge_color="red", pos=pos, width=0.6)

Vi kan bruge widgets til "nemt" at lave dashboard til at vælge gode matches at gemme :D

In [ ]:
import ipywidgets
from IPython.display import display

In [ ]:
relevant = "amt,herred,sogn,navn,fødested,fødeår,civilstand,position,erhverv".split(",")

In [ ]:
table_fmt = """<table>
<tr>
  <th>amt</th>
  <th>herred</th>
  <th>sogn</th>
  <th>navn</th>
  <th>fødested</th>
  <th>fødeår</th>
  <th>civilstand</th>
  <th>position</th>
  <th>erhverv</th>
</tr>
{}
</table>"""

In [ ]:
def to_html_row(entry):
    return "<tr>" + "".join("<td>" + str(getattr(entry, field)) + "</td>" for field in relevant)

In [ ]:
matched_pairs = []
for u, v in match:
    a_row = a_data[int(u[1:])]
    b_row = b_data[int(v[1:])]
    matched_pairs.append((G[u][v]["weight"], a_row, b_row))
matched_pairs.sort(reverse=True)

display(ipywidgets.Label(value=str(a_year) + "-" + str(b_year)))
for (w, a, b) in matched_pairs[:100]:
    if (a, b) in cache.data[(a_year, b_year)]:
        continue
    but = ipywidgets.Button(description="approve")
    def add_this_pair_maker(*args): # nested functions for closure
        def enclosed(nonsense):
            cache.add_match(*args)
        return enclosed
    but.on_click(add_this_pair_maker(a_year, b_year, a, b))
    display(ipywidgets.Label(value="w=" + str(w)))
    display(ipywidgets.HTML(table_fmt.format(to_html_row(a) + to_html_row(b))))
    display(but)

In [ ]:
cache.flush()

Bemærk: kvinder skifter navn, så vi bør smide efternavn væk på kvinder i matching.

Og vi skal have sogne som subsets af områder ift. fødested; ane marie knudsdatter født på læsøe eller byrum er lige godt.  Eller måske vesterø og læsø.

HOV! Hvorfor er der over 5mio rows i nogle af filerne? Så mange mennesker var der ikke!

Cacher lige gode matches som $(y_1,y_2) \to \{(r_1, r_2) \dots\}$